### 内容提要
> 1)获取会计科目数据；2)编制财务指标；3)显示、导出数据；4)可视化、图表


### 第一部分、获取会计科目数据

**用两种方法可以处理，这里都写出来，可以看出直接处理文件比使用Pandas性能高出很多**，另外，在由于下载的数据里面包括一些无效数据，所以处理时容易出错，不推荐使用Pandas处理数据。
注意：数据要先下载到s_file变量指定的位置，之前的gFanacials.py爬虫可以完成数据下载工作。这里是本Notebook同级的data目录，为了保证代码运行，已经把示例所需数据下载下来了。


In [8]:
# 获取会计科目数据
# 方法一、通过Pandas库从csv文件获取数据
# 写成函数，code为六位股票代码，itp为报表类型：'is for income_statement, bs for balance_sheet, cf for cash_flows
# n为默认参数，0表示最近一期，1表示最近两期，以下同
def select(code,itp,key,n=0):
    import pandas as pds
    s_file='data/{tp}{c}.csv'.format(c=code,tp=itp)
    df=pds.read_csv(s_file,encoding='gbk',index_col=0).dropna(axis=1)
    return df.ix[df.index.str.contains(key)].values[0][n]

%time select('600660','cf','经营活动产生的现金流量净额')

CPU times: user 8 ms, sys: 1e+03 µs, total: 9 ms
Wall time: 8.23 ms


'363697'

In [9]:
# 获取会计科目数据：方法二、直接处理文本
def select2(code,itp,key,n=0):
    s_file='data/{tp}{c}.csv'.format(c=code,tp=itp)
    ct=open(s_file,encoding='gbk').read().replace('--','0')
    targ = ''
    start = ct.find(key)
    end = 0
    if start>-1:
        if key[0]=='\n':
            end = ct.find('\n',start+1)
        else:
            end = ct.find('\n',start)
        targ = ct[start:end-1]
    return float(targ.split(',')[n+1])

%time select2('600660','cf','经营活动产生的现金流量净额')

CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 498 µs


363697.0


### 第二部分、编制财务指标

**其实拿到数据后要计算财务指标并导出是非常简单的事，使用一个双重循环即可。但是这可能会使得写出的代码非常不灵活，打个比方，今天可能要导出盈利能力数据。好容易将代码写好了，明天可能又需要导出其它数据。然后就到原代码基础上改改改。今天导出沪深300的，明天可能又要中证500的怎么办。我们编制财务指标，就是为了将代码模块化，方便调用，以后不管需要什么数据，只需提供范围就可以导出数据了**



In [4]:
# 这里只举一个例子并说明
# 权益乘数
def em(code,ny=0):
    return round(select2(code,'bs','资产总计',ny)/select2(code,'bs','所有者权益(或股东权益)合计',ny),2)

def roic(code,ny=0):
    i1 = select2(code,'is','净利润',ny)
    l1 = select2(code,'bs','短期借款',ny)
    l2 = select2(code,'bs','一年内到期的非流动负债',ny)
    l3 = select2(code,'bs','其他流动负债',ny)
    l4 = select2(code,'bs','长期借款',ny)
    l5 = select2(code,'bs','应付债券',ny)
    l6 = select2(code,'bs','其他非流动负债',ny)
    e1 = select2(code,'bs','所有者权益(或股东权益)合计',ny)
    return round(i1/(l1+l2+l3+l4+l5+l6+e1),2)

print('福耀玻璃的权益乘数是：', em('600660'))
print('福耀玻璃的投入资本回报率是:',roic('600660'))


福耀玻璃的权益乘数是： 1.66
福耀玻璃的投入资本回报率是: 0.12


上面两个函数为定义的两个财务指标，大家可能注意到两个函数使用了一致的调用参数。什么，你不知道什么是调用参数？那。。。这么着说吧，假设有一个快递员，他在送快递时需要一个地址和送的包裹吧。我们在“定义”送快递这个动作的时候，“地址”和“包裹”是快递员在“送包裹”时需要的东西，这个就是形参数。因为它不代表具体的地址和具体的包裹，而在实际送包裹的时候（函数调用）需要提供实际的“地址（比如：城市花园233号”和“包裹（比如：狗粮）”。这个“城市花园233号”和“狗粮”就是调用参数（又称为实际参数），现在，“将这包狗粮送到城市花园233号”就完成了一次“送快递”动作，哎说不清楚了。未明白的，去看Python的相关教程吧。

由于我们在定义函数时采用了一致的参数，这就方便了我们使用同一方式进行函数调用，从而获得相应数据。接下来的第三部分，简单说说一次性送多个包裹（哦不，获取多只股票多个数据）的方法。

### 第三部分、导出数据

接下来，我们定义一个导出数据的函数（就是功能），接收两个参数：一个是需要数据的股票列表，一个是须要导出的财务指标。

In [7]:
from collections import OrderedDict
import tushare as t
def export_data(rn=['600660','000895'],fi_dict=OrderedDict([('权益乘数',em),('投入资本回报率',roic)])):
    stock_b = t.get_stock_basics()
    stock_b = stock_b.ix[:,0:3]
    stock_b.columns=['股票名称','所在行业','所在区域']
    stock_b = stock_b.ix[rn]
    for k in fi_dict:
        values = []
        for c in stock_b.index:
            try:
                values.append(fi_dict[k](c))
            except:
                values.append(0)
        stock_b[k] = values
    #stock_b.to_csv('Fi.csv',encoding='gbk')
    return stock_b

%time export_data()

CPU times: user 43 ms, sys: 9 ms, total: 52 ms
Wall time: 432 ms


,股票名称,所在行业,所在区域,权益乘数,投入资本回报率
code,,,,,
600660,福耀玻璃,汽车配件,福建,1.66,0.12
000895,双汇发展,食品,河南,1.42,0.28


这里稍做解释：函数export_data定义了两个命名参数，“rn”为一个列表，表示所需获得数据的股票列表，默认值为['600660','000895']，fi_dict是需要导出的财务指标，这里用了一个OrderedDict是为了保证财务指标按照指定的顺序输出。
该功能返回一个pandas.DataFrame表格，如上图。
注：各位小伙伴还可以编写更多的财务指标，实际更多样化的输出（为重复工作，这里不再赘述）

### 第四部分、数据可视化及图表

坦率讲我并不喜欢通过图表展示数据，感觉能够看得懂的图表吧展示的数据量不够，数据量大的图表吧，又不容易看得懂。所以这部分就不举例子了。Matplotlib官网上提供了各种图范例，如果有需要，将代码copy过来改下调用参数就可以了。参考 [MatPlotLib Gallery][1]

P.S: 其实很多知友都在吐槽说，Wind的Excel插件那么好用，为啥要专门用Python来如何如何。各位大佬，Wind的大名小生还是知道的，不过Wind的价格。。。更何况，不是所有人都是机构投资者的，所以，稍微动动手，9999元/年的数据使用费就省下来了，有没有很惊喜。

P.P.S: 本专栏为一个财务专栏，可能很多人认为不够专注，一会儿是Python，一会又是Excel。在这里给大伙做一说明：不管是Excel还是Python都是财务分析的一个手段，我会在不同的场景下使用不同的工具，目的都是从财务报表中获得有用的数据。

再P.P.S: 本专栏开篇挖了两个坑，一个坑是网络狗读财报系列，一个坑是Python与财务分析系列。现在**PYTHON与财务系列**这个坑算是填完了，另外一个**网络狗读财报**系列也会慢慢填上的。

[1]: http://matplotlib.org/gallery.html "MatPlotLib Gallery"